In [1]:
import pandas as pd
import numpy as np

In [16]:
import tensorflow as tf
import tensorflow_hub as hub
import keras
from sklearn.model_selection import train_test_split

In [12]:
from keras.layers import Input,Lambda,Dense,Dropout
from keras.models import Model
import keras.backend as K

In [5]:
#data loading 
fake = pd.read_csv("gossipcop_fake.csv")
real = pd.read_csv("gossipcop_real.csv")
fake_t = fake.title.values
real_t = real.title.values

In [7]:
X = np.concatenate([fake_t,real_t],axis=0)

In [10]:
fake["label"] = 0
real["label"] = 1

In [11]:
fake_l = fake.label.values
real_l = real.label.values
Y = np.concatenate([fake_l,real_l],axis=0)

In [18]:
X_train,X_test,y_train,y_test = train_test_split(X,Y,test_size=0.2,random_state=42)

In [13]:
url = "https://tfhub.dev/google/universal-sentence-encoder-large/3"
embed = hub.Module(url)

INFO:tensorflow:Using C:\Users\anubh\AppData\Local\Temp\tfhub_modules to cache modules.


In [14]:
def UniversalEmbedding(x):
    return embed(tf.squeeze(tf.cast(x, tf.string)))

In [23]:
input_text = Input(shape=(1,), dtype=tf.string)
embedding = Lambda(UniversalEmbedding, output_shape=(512, ))(input_text)
drop_1 = Dropout(0.4)(embedding)
dense = Dense(256, activation='relu')(drop_1)
drop_2 = Dropout(0.2)(dense)
pred = Dense(1, activation='sigmoid')(drop_2)
model = Model(inputs=[input_text], outputs=pred)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

INFO:tensorflow:Saver not created because there are no variables in the graph to restore
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 1)                 0         
_________________________________________________________________
lambda_4 (Lambda)            (None, 512)               0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 256)               131328    
_________________________________________________________________
dropout_4 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 257       
Total params: 131,585
Trainable params: 131,585
Non-t

In [24]:
with tf.Session() as session:
    K.set_session(session)
    session.run(tf.global_variables_initializer())  
    session.run(tf.tables_initializer())
    history = model.fit(X_train, y_train, epochs=1, batch_size=32)
    model.save_weights('./model.h5')

Epoch 1/1
17712/17712 [==============================] - 167s 9ms/step - loss: 0.4530 - acc: 0.7952
